In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
import geopandas as gpd 
import os
import pandas as pd 
# ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
data_dir = r'D:\Projects\superparcels\data\urban'
parcels = gpd.read_file(os.path.join(data_dir, 'sp_sample_08013_urban.shp'))
utm = parcels.estimate_utm_crs().to_epsg()

In [ ]:
parcels = parcels.to_crs(epsg=utm)  

In [ ]:
candi = parcels[parcels['OWNER'].map(parcels['OWNER'].value_counts()) > 1]
candi['OWNER'].value_counts()

In [ ]:
candi['point'] = candi['geometry'].representative_point()
candi.head()

In [ ]:
unique_owners = candi['OWNER'].unique()
unique_owners



In [ ]:
cleaned_parcel_data = gpd.GeoDataFrame()
for owner in unique_owners:
    owner_parcels = candi[candi['OWNER'] == owner]
    owner_coords = [x.coords[0] for x in owner_parcels['point'].to_list()]
    dbscan = DBSCAN(eps=50, min_samples=1, metric='euclidean')
    clusters = dbscan.fit_predict(owner_coords)
    owner_parcels['cluster'] = clusters 
    counts = owner_parcels['cluster'].value_counts()
    single_parcel_clusters = counts[counts == 1].index
    print(single_parcel_clusters)
    
    parcel_filter = owner_parcels[~owner_parcels['cluster'].isin(single_parcel_clusters)]
    cleaned_parcel_data = pd.concat([cleaned_parcel_data, parcel_filter], ignore_index=True)
    #owner_coords = owner_parcels['point']
    

In [ ]:
cleaned_parcel_data.drop(columns='point').to_file(os.path.join(data_dir, 'cleaned_canidate_parcels.shp'))

In [ ]:
owner_parcels.drop(columns='point').to_file(os.path.join(data_dir, 'owner_parcels.shp'))

In [ ]:
owner_coords

In [ ]:

# Define DBSCAN parameters (adjust eps based on your distance criteria)
dbscan = DBSCAN(eps=1200, min_samples=1, metric='euclidean')  # Adjust eps for your distance threshold
clusters = dbscan.fit_predict(owner_coords)



In [ ]:
#clusters = np.array([0,1,1,0,2])
clusters.value_counts()

In [ ]:
# determine which cluster ids occurece more than once
unique, counts = np.unique(clusters, return_counts=True)
dup_clusters = unique[counts == 1]
dup_clusters


In [ ]:
for candi_cluster in dup_clusters:
    # drop parcel that is associated with the cluster
    owner_parcels = owner_parcels[clusters != candi_cluster]

In [ ]:
owner_parcels